# Active SIMs forecast

In [ ]:
# relevant import

import pandas as pd
import datetime
from dateutil import relativedelta
from datetime import date, timedelta
from datetime import datetime
import numpy as np

In [ ]:
#define the number of past months for which we want to average to calculate the percentage of returning sims
backwards=3

In [ ]:
starting_date_for_pred=date(2023, 5, 1)
end_date=date(2024, 1, 1)
date_for_pred= date(2023,6,1)

starting_date_for_pred

datetime.date(2023, 5, 1)

In [ ]:
# Specify variables based on which to segment

segment=['vertical_group','country_group','initial_1k_sims']

## Preprocessing

Importing the monthly usage table from weld

In [ ]:
sim = _deepnote_execute_sql('with\n    monthly_usage_with_previous as (\n        select\n            t1.organization_id\n          , t1.sim_id\n          , t1.usage_month as month\n          , t2.assigned_month\n          , t2.activated_month\n          , t2.deployed_3_months as deployed_month\n          , lag(usage_month) over (\n                partition by t1.sim_id\n                order by\n                    usage_month\n            ) as previous_month_online\n          , lead(usage_month) over (\n                partition by t1.sim_id\n                order by\n                    usage_month\n            ) as next_month_online\n          , date_diff(usage_month, assigned_month, month) as months_since_assigned\n        from\n            `staging`.`postgresql_monthly_sim_usage` t1\n            left join `core`.`sims__sim_lifecycle` t2 on t1.sim_id = t2.sim_id\n    )\n  , final as (\n        select\n            *\n          , date_diff(month, previous_month_online, month) as months_since_last_online\n          , date_diff(next_month_online, month, month) as months_until_next_online\n        from\n            monthly_usage_with_previous\n    )\nselect\n    *\nfrom\n    final', 'SQL_589DF268_08DE_4FD7_9575_ADF75AFF2D19', audit_sql_comment='', sql_cache_mode='cache_disabled')
sim

,organization_id,sim_id,month,assigned_month,activated_month,deployed_month,previous_month_online,next_month_online,months_since_assigned,months_since_last_online,months_until_next_online
0,19,000006200,2019-07-01,2019-10-01,2019-06-01,2019-06-01,2019-06-01,2019-08-01,-3,1.0,1.0
1,19,000006200,2019-08-01,2019-10-01,2019-06-01,2019-06-01,2019-07-01,2019-09-01,-2,1.0,1.0
2,19,000006200,2019-09-01,2019-10-01,2019-06-01,2019-06-01,2019-08-01,2019-10-01,-1,1.0,1.0
3,19,000006200,2019-10-01,2019-10-01,2019-06-01,2019-06-01,2019-09-01,2019-11-01,0,1.0,1.0
4,19,000006200,2019-11-01,2019-10-01,2019-06-01,2019-06-01,2019-10-01,2019-12-01,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4151743,409,000249529,2021-04-01,2020-11-01,2021-03-01,2021-03-01,2021-03-01,2021-05-01,5,1.0,1.0
4151744,627,000270205,2021-04-01,2021-01-01,2021-01-01,2021-01-01,2021-03-01,2021-05-01,3,1.0,1.0
4151745,591,000271564,2021-04-01,2021-02-01,2021-03-01,None,2021-03-01,None,2,1.0,NaN
4151746,775,000271606,2021-04-01,2021-02-01,2021-03-01,None,2021-03-01,None,2,1.0,NaN


removing test organizations

In [ ]:
sim[(sim.organization_id==28) | (sim.organization_id==479)]

,organization_id,sim_id,month,assigned_month,activated_month,deployed_month,previous_month_online,next_month_online,months_since_assigned,months_since_last_online,months_until_next_online
2242,28,000071084,2020-08-01,2019-10-01,2020-07-01,2020-07-01,2020-07-01,2020-09-01,10,1.0,1.0
2243,28,000071084,2020-09-01,2019-10-01,2020-07-01,2020-07-01,2020-08-01,2020-10-01,11,1.0,1.0
2244,28,000071084,2020-10-01,2019-10-01,2020-07-01,2020-07-01,2020-09-01,2020-11-01,12,1.0,1.0
2245,28,000071084,2020-11-01,2019-10-01,2020-07-01,2020-07-01,2020-10-01,2020-12-01,13,1.0,1.0
2246,28,000071148,2019-10-01,2019-10-01,2019-09-01,2019-09-01,2019-09-01,2019-11-01,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4151737,28,000230039,2021-06-01,2020-09-01,2021-03-01,2021-06-01,2021-03-01,2021-07-01,9,3.0,1.0
4151738,28,000230207,2021-07-01,2020-09-01,2021-03-01,2021-11-01,2021-03-01,2021-08-01,10,4.0,1.0
4151739,28,000230270,2021-04-01,2020-09-01,2021-03-01,2021-06-01,2021-03-01,2021-06-01,7,1.0,2.0
4151740,28,000230311,2021-04-01,2020-09-01,2021-03-01,2021-03-01,2021-03-01,2021-05-01,7,1.0,1.0


In [ ]:
sim=sim[(sim.organization_id!=1.0) & (sim.organization_id!=2.0)]

In [ ]:
sim=sim[(sim.organization_id!=229) & (sim.organization_id!=406)]

In [ ]:
sim=sim[(sim.organization_id!=412) & (sim.organization_id!=1109)]

In [ ]:
sim=sim[(sim.organization_id!=229) & (sim.organization_id!=1066)]

In [ ]:
sim.sim_id=sim.sim_id.astype('int')

In [ ]:
sim

,organization_id,sim_id,month,assigned_month,activated_month,deployed_month,previous_month_online,next_month_online,months_since_assigned,months_since_last_online,months_until_next_online
0,19,6200,2019-07-01,2019-10-01,2019-06-01,2019-06-01,2019-06-01,2019-08-01,-3,1.0,1.0
1,19,6200,2019-08-01,2019-10-01,2019-06-01,2019-06-01,2019-07-01,2019-09-01,-2,1.0,1.0
2,19,6200,2019-09-01,2019-10-01,2019-06-01,2019-06-01,2019-08-01,2019-10-01,-1,1.0,1.0
3,19,6200,2019-10-01,2019-10-01,2019-06-01,2019-06-01,2019-09-01,2019-11-01,0,1.0,1.0
4,19,6200,2019-11-01,2019-10-01,2019-06-01,2019-06-01,2019-10-01,2019-12-01,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4151743,409,249529,2021-04-01,2020-11-01,2021-03-01,2021-03-01,2021-03-01,2021-05-01,5,1.0,1.0
4151744,627,270205,2021-04-01,2021-01-01,2021-01-01,2021-01-01,2021-03-01,2021-05-01,3,1.0,1.0
4151745,591,271564,2021-04-01,2021-02-01,2021-03-01,None,2021-03-01,None,2,1.0,NaN
4151746,775,271606,2021-04-01,2021-02-01,2021-03-01,None,2021-03-01,None,2,1.0,NaN


In [ ]:
# benchmark for comparison

may=date(2023, 6, 1)
sim[sim.month==may].sim_id.count()

264017

adding info from segmentation on a company level

In [ ]:
pip install openpyxl

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
companies=pd.read_excel('/work/ad_hoc_analysis.customer_segmentation.customer_segmentation_prep.xlsx')

In [ ]:
# removing duplicates
companies=companies.drop_duplicates(subset=['organization_id'])


Merge info at company level with info at sim level

In [ ]:
# add info at company level
df=sim.merge(companies, on='organization_id')


In [ ]:
# keeping only entries related to an organization
df=df[df.organization_id.isna()==False]

In [ ]:
df.columns

Index(['organization_id', 'sim_id', 'month', 'assigned_month',
       'activated_month', 'deployed_month', 'previous_month_online',
       'next_month_online', 'months_since_assigned',
       'months_since_last_online', 'months_until_next_online', 'company_id',
       'deal_id', 'company_name', 'number_of_employees',
       'potential_sims_group', 'vertical_group', 'country_group',
       'solution_transparency_group', 'first_deal_velocity', 'close_date',
       'first_platform_fee', 'months', 'max_active_SIMs',
       'active_SIMs_divided_by_months', 'active_SIMs_3', 'active_SIMs_6',
       'active_SIMs_9', 'active_SIMs_12', 'mrr_3', 'mrr_6', 'mrr_9', 'mrr_12'],
      dtype='object')

adding information about sims being in 1st 1k or not

In [ ]:
for_initial_1k = _deepnote_execute_sql('with\n    sim_lifecycle as (\n        select\n            *\n        from\n            `core`.`sims__sim_lifecycle`\n    )\n  , mud_last_month as (\n        select\n            distinct sim_id\n        from\n            `staging`.`postgresql_monthly_sim_usage`\n        where\n            date_diff(date_trunc(current_date, month), usage_month, month) = 1\n    )\n  , mud_prev_month as (\n        select\n            distinct sim_id\n        from\n            `staging`.`postgresql_monthly_sim_usage`\n        where\n            date_diff(date_trunc(current_date, month), usage_month, month) = 2\n    )\n  , combined as (\n        select\n            sim_lifecycle.*\n          , mud_last_month.sim_id is not null as active_last_month\n          , mud_prev_month.sim_id is not null as active_prev_month\n      , from\n            sim_lifecycle\n            left join mud_last_month on sim_lifecycle.sim_id = mud_last_month.sim_id\n            left join mud_prev_month on sim_lifecycle.sim_id = mud_prev_month.sim_id\n    )\nselect\n    *\nfrom\n    combined', 'SQL_589DF268_08DE_4FD7_9575_ADF75AFF2D19', audit_sql_comment='', sql_cache_mode='cache_disabled')
for_initial_1k

,sim_id,organization_id,activated_month,assigned_month,deployed_2_months,deployed_3_months,dead_month_2,months_online_since_deployed_2,dead_month_3,months_online_since_deployed_3,initial_1k_sims,initial_set_of_sims,last_online_month_2,last_online_month_3,active_last_month,active_prev_month
0,000001166,2.0,2019-01-01,2019-10-01,2019-03-01,2019-03-01,2019-06-01,3.0,2019-06-01,3.0,True,True,2019-05-01,2019-05-01,False,False
1,000001214,25.0,2019-08-01,2019-10-01,2019-08-01,2019-08-01,None,NaN,None,NaN,True,True,None,None,True,True
2,000001291,28.0,None,2019-10-01,None,None,None,NaN,None,NaN,True,True,None,None,False,False
3,000001485,NaN,None,None,None,None,None,NaN,None,NaN,True,None,None,None,False,False
4,000001521,28.0,None,2019-11-01,None,None,None,NaN,None,NaN,False,False,None,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193540,001374137,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False
1193541,001374226,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False
1193542,001374245,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False
1193543,001374275,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False


In [ ]:
#df[df.activated_month<start_date]

In [ ]:
# keep data from 2019 onwards 

start_date=date(2019, 1, 1)
df=df[df.activated_month>start_date]

In [ ]:
# creating df that will be needed later

sim_lifecycle=for_initial_1k.copy()

In [ ]:
for_initial_1k=for_initial_1k[['sim_id','initial_1k_sims']]

In [ ]:
for_initial_1k.sim_id=for_initial_1k.sim_id.astype('int')

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df=df.merge(for_initial_1k, on='sim_id')

In [ ]:
df.head()

,organization_id,sim_id,month,assigned_month,activated_month,deployed_month,previous_month_online,next_month_online,months_since_assigned,months_since_last_online,...,active_SIMs_divided_by_months,active_SIMs_3,active_SIMs_6,active_SIMs_9,active_SIMs_12,mrr_3,mrr_6,mrr_9,mrr_12,initial_1k_sims
0,19,6200,2019-07-01,2019-10-01,2019-06-01,2019-06-01,2019-06-01,2019-08-01,-3,1.0,...,957.388889,6962.0,16551.0,15667.0,14267.0,708.8,1671.1,1582.5,1440.7,True
1,19,6200,2019-08-01,2019-10-01,2019-06-01,2019-06-01,2019-07-01,2019-09-01,-2,1.0,...,957.388889,6962.0,16551.0,15667.0,14267.0,708.8,1671.1,1582.5,1440.7,True
2,19,6200,2019-09-01,2019-10-01,2019-06-01,2019-06-01,2019-08-01,2019-10-01,-1,1.0,...,957.388889,6962.0,16551.0,15667.0,14267.0,708.8,1671.1,1582.5,1440.7,True
3,19,6200,2019-10-01,2019-10-01,2019-06-01,2019-06-01,2019-09-01,2019-11-01,0,1.0,...,957.388889,6962.0,16551.0,15667.0,14267.0,708.8,1671.1,1582.5,1440.7,True
4,19,6200,2019-11-01,2019-10-01,2019-06-01,2019-06-01,2019-10-01,2019-12-01,1,1.0,...,957.388889,6962.0,16551.0,15667.0,14267.0,708.8,1671.1,1582.5,1440.7,True


Get an idea of number of SIMs assigned per month over time

In [ ]:
# sims assigned per month
avg_ass=df.groupby('assigned_month').count()
avg_ass.reset_index(inplace=True)

In [ ]:
# overall average
avg_ass.sim_id.mean()

91910.13333333333

In [ ]:
# average for year 2023
month_for_avg=date(2023, 1, 1)
avg_ass=avg_ass[avg_ass.assigned_month>=month_for_avg]
avg_ass.sim_id.mean()

16982.0

### Data Preparation

get number of sims per each segment per month per month_since_last_online

In [ ]:
# create dataset grouped by segment and by month
base= ['month','months_since_last_online']
base.extend(segment)
segmentation=base
segmented_by_month=df.groupby(segmentation).count()

In [ ]:
segmented_by_month
segmented_by_month.reset_index(inplace=True)

In [ ]:
#segmented_by_month[segmented_by_month.month==may]

In [ ]:
segmented_df=segmented_by_month.copy()

In [ ]:
#making a touple for the segment for easier retrieval later
segmented_by_month['segment']=segmented_by_month[segment].apply(tuple, axis=1)

In [ ]:
segmented_by_month.head()

,month,months_since_last_online,vertical_group,country_group,initial_1k_sims,organization_id,sim_id,assigned_month,activated_month,deployed_month,...,active_SIMs_divided_by_months,active_SIMs_3,active_SIMs_6,active_SIMs_9,active_SIMs_12,mrr_3,mrr_6,mrr_9,mrr_12,segment
0,2019-03-01,1.0,Vertical Group 1,Country Group 1,True,3,3,3,3,0,...,3,3,3,3,3,3,3,3,3,"(Vertical Group 1, Country Group 1, True)"
1,2019-03-01,1.0,Vertical Group Other,Country Group 1,True,2,2,2,2,2,...,2,1,2,2,2,1,2,2,2,"(Vertical Group Other, Country Group 1, True)"
2,2019-04-01,1.0,Vertical Group 1,Country Group 1,False,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,"(Vertical Group 1, Country Group 1, False)"
3,2019-04-01,1.0,Vertical Group 1,Country Group 1,True,2,2,2,2,1,...,2,2,2,2,2,2,2,2,2,"(Vertical Group 1, Country Group 1, True)"
4,2019-04-01,1.0,Vertical Group 2,Country Group 1,True,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,"(Vertical Group 2, Country Group 1, True)"


Get total amount per sims in each segment by by month to obtain the total to compare with

In [ ]:
base= ['month']
base.extend(segment)
segmentation=base

In [ ]:
#grouping by the 3 features for which we segment and by "month"

tot_per_segment=df.groupby(base).count()
tot_per_segment.reset_index(inplace=True)


In [ ]:
#making a touple for the segment for easier retrieval later

tot_per_segment['segment']=tot_per_segment[segment].apply(tuple, axis=1)

In [ ]:
tot_per_segment.head()

,month,vertical_group,country_group,initial_1k_sims,organization_id,sim_id,assigned_month,activated_month,deployed_month,previous_month_online,...,active_SIMs_divided_by_months,active_SIMs_3,active_SIMs_6,active_SIMs_9,active_SIMs_12,mrr_3,mrr_6,mrr_9,mrr_12,segment
0,2019-02-01,Vertical Group 1,Country Group 1,True,3,3,3,3,0,0,...,3,3,3,3,3,3,3,3,3,"(Vertical Group 1, Country Group 1, True)"
1,2019-02-01,Vertical Group 3,Country Group Other,True,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,"(Vertical Group 3, Country Group Other, True)"
2,2019-02-01,Vertical Group Other,Country Group 1,True,3,3,3,3,3,0,...,3,1,3,3,3,1,3,3,3,"(Vertical Group Other, Country Group 1, True)"
3,2019-03-01,Vertical Group 1,Country Group 1,False,18,18,18,18,6,0,...,18,18,18,18,18,18,18,18,18,"(Vertical Group 1, Country Group 1, False)"
4,2019-03-01,Vertical Group 1,Country Group 1,True,7,7,7,7,1,3,...,7,7,7,7,7,7,7,7,7,"(Vertical Group 1, Country Group 1, True)"


In [ ]:
#renaming --> segment_total: n. of sims per segment per month
tot_per_segment=tot_per_segment.rename(columns={'sim_id':'segment_total'})


Adding the info about total sims per segment to the segmented dataset

In [ ]:
# joining info about total sims per segment and grouped dataframe
 
final=segmented_by_month.merge(tot_per_segment[['segment','segment_total','month']], on=['month','segment'])

In [ ]:
final=final.rename(columns={'sim_id':'sim_count'})

In [ ]:
final.sim_count.sum()

3670938

get returning percentage per segment/returning month/activated month

In [ ]:
# (n. of sims per segment per month per month_since_last_online)/(n. of sims per segment per month)

final['percentage_back']=final['sim_count']/final['segment_total']

In [ ]:
final.head()

,month,months_since_last_online,vertical_group,country_group,initial_1k_sims,organization_id,sim_count,assigned_month,activated_month,deployed_month,...,active_SIMs_6,active_SIMs_9,active_SIMs_12,mrr_3,mrr_6,mrr_9,mrr_12,segment,segment_total,percentage_back
0,2019-03-01,1.0,Vertical Group 1,Country Group 1,True,3,3,3,3,0,...,3,3,3,3,3,3,3,"(Vertical Group 1, Country Group 1, True)",7,0.428571
1,2019-03-01,1.0,Vertical Group Other,Country Group 1,True,2,2,2,2,2,...,2,2,2,1,2,2,2,"(Vertical Group Other, Country Group 1, True)",2,1.000000
2,2019-04-01,1.0,Vertical Group 1,Country Group 1,False,4,4,4,4,4,...,4,4,4,4,4,4,4,"(Vertical Group 1, Country Group 1, False)",12,0.333333
3,2019-04-01,1.0,Vertical Group 1,Country Group 1,True,2,2,2,2,1,...,2,2,2,2,2,2,2,"(Vertical Group 1, Country Group 1, True)",2,1.000000
4,2019-04-01,1.0,Vertical Group 2,Country Group 1,True,1,1,1,1,1,...,1,1,1,1,1,1,1,"(Vertical Group 2, Country Group 1, True)",2,0.500000


In [ ]:
#keeping only relevant columns
relevant_cols=['month','months_since_last_online','sim_count','segment_total','segment','percentage_back']
relevant_cols.extend(segment)
final=final[relevant_cols]

In [ ]:
final

,month,months_since_last_online,sim_count,segment_total,segment,percentage_back,vertical_group,country_group,initial_1k_sims
0,2019-03-01,1.0,3,7,"(Vertical Group 1, Country Group 1, True)",0.428571,Vertical Group 1,Country Group 1,True
1,2019-03-01,1.0,2,2,"(Vertical Group Other, Country Group 1, True)",1.000000,Vertical Group Other,Country Group 1,True
2,2019-04-01,1.0,4,12,"(Vertical Group 1, Country Group 1, False)",0.333333,Vertical Group 1,Country Group 1,False
3,2019-04-01,1.0,2,2,"(Vertical Group 1, Country Group 1, True)",1.000000,Vertical Group 1,Country Group 1,True
4,2019-04-01,1.0,1,2,"(Vertical Group 2, Country Group 1, True)",0.500000,Vertical Group 2,Country Group 1,True
...,...,...,...,...,...,...,...,...,...
5559,2023-07-01,1.0,834,835,"(Vertical Group Other, Country Group 2, True)",0.998802,Vertical Group Other,Country Group 2,True
5560,2023-07-01,3.0,1,835,"(Vertical Group Other, Country Group 2, True)",0.001198,Vertical Group Other,Country Group 2,True
5561,2023-07-01,1.0,45,45,"(Vertical Group Other, Country Group Other, Fa...",1.000000,Vertical Group Other,Country Group Other,False
5562,2023-07-01,1.0,1162,1163,"(Vertical Group Other, Country Group Other, True)",0.999140,Vertical Group Other,Country Group Other,True


---


## Returning SIMs % calculation

In [ ]:
#calculate n. of months between date and prediction date

def count_months(date_for_pred,date):
    r = relativedelta.relativedelta(date_for_pred,date)    
    months = (r.years * 12) + r.months
    return months 

In [ ]:
#change to date type ---> only needed whten uploading a csv as the date is read as string
#final.month=final.month.apply(pd.to_datetime)

In [ ]:
#calculate n.of month between "month" and the month for which we are calculating returning sims

final['month_diff'] = final.month.apply(lambda x:count_months(starting_date_for_pred,x))

In [ ]:
final.head()

,month,months_since_last_online,sim_count,segment_total,segment,percentage_back,vertical_group,country_group,initial_1k_sims,month_diff
0,2019-03-01,1.0,3,7,"(Vertical Group 1, Country Group 1, True)",0.428571,Vertical Group 1,Country Group 1,True,50
1,2019-03-01,1.0,2,2,"(Vertical Group Other, Country Group 1, True)",1.000000,Vertical Group Other,Country Group 1,True,50
2,2019-04-01,1.0,4,12,"(Vertical Group 1, Country Group 1, False)",0.333333,Vertical Group 1,Country Group 1,False,49
3,2019-04-01,1.0,2,2,"(Vertical Group 1, Country Group 1, True)",1.000000,Vertical Group 1,Country Group 1,True,49
4,2019-04-01,1.0,1,2,"(Vertical Group 2, Country Group 1, True)",0.500000,Vertical Group 2,Country Group 1,True,49


the percentage for which we are averaging are now different and close to the number we were expecting (at least for the first months)

In [ ]:
# create dictionary where to store the percentage of returning sims
return_pct={}
for i in range(1,36):
    for s in final.segment.unique():
        return_pct[i,s]=0


In [ ]:
returns={}
tot=0

#range: looking at returns % after m months, from 1 to 12 (no activity after 12 months)
for m in range(1,12): #capping at 12 as after that % are very low
    temp=final[final.months_since_last_online==m] #looking at returns after m months
    temp= temp[(temp.month_diff>=m)&(temp.month_diff<m+backwards)] #selecting the n. of previous months based on the "backwards" variable
    
   #print(temp)
    for s in final.segment.unique(): #doing the calculation for each segment
        temp_segment=temp[temp.segment==s]
        temp_segment.reset_index(inplace=True)
        #print(temp_segment)
        avg_pct=np.nanmean(temp_segment.percentage_back)
        #print(avg_pct)
        return_pct[m,s]=avg_pct 
        # retireving avg percentage if existing for that segment
        try:
            avg=temp_segment.sim_count.iloc[0]
        except:
            avg=0
        tot_activation= avg*avg_pct #total sims activated (returning)

        if tot_activation>=0:
            
            tot+=tot_activation #add to running total
        returns[starting_date_for_pred,s,m]=tot_activation #add to dictionary

/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)
/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)
/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)
/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)
/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)
/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)
/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)
/tmp/ipykernel_1557/1950192396.py:14: RuntimeWarning: Mean of empty slice
  avg_pct=np.nanmean(temp_segment.percentage_back)


In [ ]:
return_pct

{(1, ('Vertical Group 1', 'Country Group 1', True)): 0.9058627897922733,
 (1, ('Vertical Group Other', 'Country Group 1', True)): 0.6858376559742264,
 (1, ('Vertical Group 1', 'Country Group 1', False)): 0.9461036585320289,
 (1, ('Vertical Group 2', 'Country Group 1', True)): 0.9080121620959561,
 (1, ('Vertical Group 3', 'Country Group 1', True)): 0.8131572361728902,
 (1, ('Vertical Group 3', 'Country Group Other', True)): 0.7859960157447156,
 (1, ('Vertical Group 2', 'Country Group 1', False)): 0.813946410595559,
 (1,
  ('Vertical Group Other', 'Country Group Other', True)): 0.7626222362377627,
 (1, ('Vertical Group Other', 'Country Group 2', True)): 0.7553625571846604,
 (1, ('Vertical Group 2', 'Country Group Other', True)): 0.8284997233621585,
 (1, ('Vertical Group Other', 'Country Group 1', False)): 0.7148439345170611,
 (1, ('Vertical Group 3', 'Country Group Other', False)): 0.8444996737511642,
 (1,
  ('Vertical Group Other', 'Country Group Other', False)): 0.7096774193548386,
 (1

create a dataframe keeping track of active sims by month

In [ ]:
# group by month and segment
base= ['month']
base.extend(segment)
segmentation=base
active_df=df.groupby(segmentation).count()
active_df.reset_index(inplace=True)

In [ ]:
active_df['segment']=active_df[segment].apply(tuple, axis=1)

In [ ]:
active_df

,month,vertical_group,country_group,initial_1k_sims,organization_id,sim_id,assigned_month,activated_month,deployed_month,previous_month_online,...,active_SIMs_divided_by_months,active_SIMs_3,active_SIMs_6,active_SIMs_9,active_SIMs_12,mrr_3,mrr_6,mrr_9,mrr_12,segment
0,2019-02-01,Vertical Group 1,Country Group 1,True,3,3,3,3,0,0,...,3,3,3,3,3,3,3,3,3,"(Vertical Group 1, Country Group 1, True)"
1,2019-02-01,Vertical Group 3,Country Group Other,True,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,"(Vertical Group 3, Country Group Other, True)"
2,2019-02-01,Vertical Group Other,Country Group 1,True,3,3,3,3,3,0,...,3,1,3,3,3,1,3,3,3,"(Vertical Group Other, Country Group 1, True)"
3,2019-03-01,Vertical Group 1,Country Group 1,False,18,18,18,18,6,0,...,18,18,18,18,18,18,18,18,18,"(Vertical Group 1, Country Group 1, False)"
4,2019-03-01,Vertical Group 1,Country Group 1,True,7,7,7,7,1,3,...,7,7,7,7,7,7,7,7,7,"(Vertical Group 1, Country Group 1, True)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,2023-07-01,Vertical Group Other,Country Group 1,True,1442,1442,1442,1442,1219,1442,...,1348,1316,1040,738,636,1316,1040,738,636,"(Vertical Group Other, Country Group 1, True)"
869,2023-07-01,Vertical Group Other,Country Group 2,False,312,312,312,312,217,312,...,312,312,312,312,9,312,312,312,9,"(Vertical Group Other, Country Group 2, False)"
870,2023-07-01,Vertical Group Other,Country Group 2,True,835,835,835,835,728,835,...,834,834,611,611,110,834,611,611,110,"(Vertical Group Other, Country Group 2, True)"
871,2023-07-01,Vertical Group Other,Country Group Other,False,45,45,45,45,27,45,...,45,45,45,0,4,45,45,0,4,"(Vertical Group Other, Country Group Other, Fa..."


In [ ]:
# removing entries after the first month predicted, to avoid dubling entries when predictions are made
active_df=active_df[active_df.month<starting_date_for_pred]

---

## Returning SIMs for last month before predictions

In [ ]:
# calculate n. of months between date and date for calculation
active_df['month_diff'] = active_df.month.apply(lambda x:count_months(starting_date_for_pred,x))

/tmp/ipykernel_1557/2318809770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_df['month_diff'] = active_df.month.apply(lambda x:count_months(starting_date_for_pred,x))


In [ ]:
active_df

,month,vertical_group,country_group,initial_1k_sims,organization_id,sim_id,assigned_month,activated_month,deployed_month,previous_month_online,...,active_SIMs_3,active_SIMs_6,active_SIMs_9,active_SIMs_12,mrr_3,mrr_6,mrr_9,mrr_12,segment,month_diff
0,2019-02-01,Vertical Group 1,Country Group 1,True,3,3,3,3,0,0,...,3,3,3,3,3,3,3,3,"(Vertical Group 1, Country Group 1, True)",51
1,2019-02-01,Vertical Group 3,Country Group Other,True,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,"(Vertical Group 3, Country Group Other, True)",51
2,2019-02-01,Vertical Group Other,Country Group 1,True,3,3,3,3,3,0,...,1,3,3,3,1,3,3,3,"(Vertical Group Other, Country Group 1, True)",51
3,2019-03-01,Vertical Group 1,Country Group 1,False,18,18,18,18,6,0,...,18,18,18,18,18,18,18,18,"(Vertical Group 1, Country Group 1, False)",50
4,2019-03-01,Vertical Group 1,Country Group 1,True,7,7,7,7,1,3,...,7,7,7,7,7,7,7,7,"(Vertical Group 1, Country Group 1, True)",50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,2023-04-01,Vertical Group Other,Country Group 1,True,1907,1907,1907,1907,1545,1671,...,1804,1436,944,952,1804,1436,944,952,"(Vertical Group Other, Country Group 1, True)",1
797,2023-04-01,Vertical Group Other,Country Group 2,False,424,424,424,424,260,308,...,424,368,368,12,424,368,368,12,"(Vertical Group Other, Country Group 2, False)",1
798,2023-04-01,Vertical Group Other,Country Group 2,True,974,974,974,974,873,892,...,973,725,725,106,973,725,725,106,"(Vertical Group Other, Country Group 2, True)",1
799,2023-04-01,Vertical Group Other,Country Group Other,False,31,31,31,31,13,4,...,31,31,0,4,31,31,0,4,"(Vertical Group Other, Country Group Other, Fa...",1


In [ ]:
# create dictionary to store activated sims later
returning={}
for i in active_df.segment.unique():
        returning[str(starting_date_for_pred),i]=0.0

In [ ]:
total=0
for month in range(1,12):
    print('returning after', month)
    temp=active_df[active_df.month_diff==month]
    for s in final.segment.unique(): #doing the calculation for each segment
        temp_segment=temp[temp.segment==s]
        try:
            sims= float(temp_segment.sim_id)
        except:
            sims=0
        pct=return_pct[month,s]
        if np.isnan(pct):
            pct=0
        ret=sims*pct
        print(ret)
        #if month==1:
        returning[str(starting_date_for_pred),s]+=ret

        total+=ret
        

returning after 1
4211.356109744279
1307.8924099428498
55632.78732900036
1602.6414660993626
2176.8219212348267
1302.3953980889937
68538.35750419904
860.2378824761963
735.7231306978592
1684.339937595268
3219.657081064843
2228.6346390293224
21.999999999999996
6711.452077586583
1613.2314111016813
7418.767301712656
472.0254893933191
29635.611050739615
577.3308898769251
1085.5490081959915
60.578743961352664
142.27353531609995
255.83682765198233
824.3212984956452
returning after 2
64.01306795335746
61.64197886423438
419.74469041465335
35.661055566891115
63.82444913631059
49.865539332229645
3529.5636897574764
13.435354340012426
29.797330306449965
55.880489954072566
197.9861536495578
33.80772580047758
0.0
104.60810593929057
18.654719008670448
419.5406510174583
50.65269797118423
735.3351966317886
19.079376815421178
25.614677581502225
5.564454614797864
6.586046511627907
35.97551873413943
94.55555555555556
returning after 3
83.04152469163144
25.461375819858876
233.6630698860602
15.487881698111257

Add new calculated month to the dataframe

In [ ]:
#new_month=pd.DataFrame.from_dict(returning, orient='index')


In [ ]:
# crate dataframe from dictionary 
new_month = (pd.DataFrame.from_dict(returning, orient='index')
                  .rename(columns=lambda x: x)
                  .add_prefix('sim_id')
                  .rename_axis('date_segment')
                  .reset_index())

In [ ]:
# split the date_segment column in 2
new_month[['month','segment']] = pd.DataFrame(new_month.date_segment.tolist(), index= new_month.index)


In [ ]:
# rename columns

new_month.rename(columns={'sim_id0':'sim_id'}, inplace=True)
new_month.drop(['date_segment'], axis=1, inplace=True)

In [ ]:
# change format to date
new_month.month = pd.to_datetime(new_month.month, format='%Y-%m-%d')
new_month.month=new_month.month.dt.date

In [ ]:
# concatenate to active_df (data for the new month)
active_df=pd.concat([active_df, new_month],ignore_index = True)#.reindex(active_new.columns, axis='columns')


---

## First activation calculations

In [ ]:
# import first activation probabilities obtained in the "activation_probs" notebook
probs=pd.read_csv('activation_probs_segm.csv')

In [ ]:
probs['id'] = probs['id'].apply(lambda x:eval(x))

In [ ]:
# create a nested dictionary for each segment, containing the percentage of sims activated in each month

probs_dict = probs.groupby('id')[['month','pct']].apply(lambda x: x.set_index('month').to_dict(orient='index')).to_dict()

In [ ]:
probs_dict

{('Vertical Group 1',
  'Country Group 1',
  False): {0: {'pct': 0.0004486594365456}, 1: {'pct': 0.021050024764735}, 2: {'pct': 0.0434084872248924}, 3: {'pct': 0.0379653733316805}, 4: {'pct': 0.0311998007061188}, 5: {'pct': 0.0247120474556433}, 6: {'pct': 0.0117735835333478}, 7: {'pct': 0.0084546951617799}, 8: {'pct': 0.0061515337081779}, 9: {'pct': 0.0147318302758717}, 10: {'pct': 0.0182021471142821}, 11: {'pct': 0.0109828597899938}, 12: {'pct': 0.0082749468178535}, 13: {'pct': 0.0031683558004526}, 14: {'pct': 0.0062581434832392}, 15: {'pct': 0.0002979915370403}, 16: {'pct': 0.0005365446524382}, 17: {'pct': 0.0005567153792623}, 18: {'pct': 0.0001392563709789}, 19: {'pct': 0.000208913649025}, 20: {'pct': 2.9851043293963123e-05}},
 ('Vertical Group 1',
  'Country Group 1',
  True): {0: {'pct': 0.0077291439132991}, 1: {'pct': 0.0405554144441622}, 2: {'pct': 0.0514472290857747}, 3: {'pct': 0.1844822774211554}, 4: {'pct': 0.0662788044718229}, 5: {'pct': 0.0612095296273671}, 6: {'pct': 0.12

In [ ]:
#sim_lifecycle=for_initial_1k.copy()

In [ ]:
sims_lifecycle = _deepnote_execute_sql('with\n    sim_lifecycle as (\n        select\n            *\n        from\n            `core`.`sims__sim_lifecycle`\n    )\n  , mud_last_month as (\n        select\n            distinct sim_id\n        from\n            `staging`.`postgresql_monthly_sim_usage`\n        where\n            date_diff(date_trunc(current_date, month), usage_month, month) = 1\n    )\n  , mud_prev_month as (\n        select\n            distinct sim_id\n        from\n            `staging`.`postgresql_monthly_sim_usage`\n        where\n            date_diff(date_trunc(current_date, month), usage_month, month) = 2\n    )\n  , combined as (\n        select\n            sim_lifecycle.*\n          , mud_last_month.sim_id is not null as active_last_month\n          , mud_prev_month.sim_id is not null as active_prev_month\n      , from\n            sim_lifecycle\n            left join mud_last_month on sim_lifecycle.sim_id = mud_last_month.sim_id\n            left join mud_prev_month on sim_lifecycle.sim_id = mud_prev_month.sim_id\n    )\nselect\n    *\nfrom\n    combined', 'SQL_589DF268_08DE_4FD7_9575_ADF75AFF2D19', audit_sql_comment='', sql_cache_mode='cache_disabled')
sims_lifecycle

,sim_id,organization_id,activated_month,assigned_month,deployed_2_months,deployed_3_months,dead_month_2,months_online_since_deployed_2,dead_month_3,months_online_since_deployed_3,initial_1k_sims,initial_set_of_sims,last_online_month_2,last_online_month_3,active_last_month,active_prev_month
0,000001166,2.0,2019-01-01,2019-10-01,2019-03-01,2019-03-01,2019-06-01,3.0,2019-06-01,3.0,True,True,2019-05-01,2019-05-01,False,False
1,000001214,25.0,2019-08-01,2019-10-01,2019-08-01,2019-08-01,None,NaN,None,NaN,True,True,None,None,True,True
2,000001291,28.0,None,2019-10-01,None,None,None,NaN,None,NaN,True,True,None,None,False,False
3,000001485,NaN,None,None,None,None,None,NaN,None,NaN,True,None,None,None,False,False
4,000001521,28.0,None,2019-11-01,None,None,None,NaN,None,NaN,False,False,None,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193540,001374137,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False
1193541,001374226,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False
1193542,001374245,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False
1193543,001374275,NaN,None,None,None,None,None,NaN,None,NaN,False,None,None,None,False,False


Removing test organizations

In [ ]:
sim_lifecycle=sim_lifecycle[(sim_lifecycle.organization_id!=1.0) & (sim_lifecycle.organization_id!=2.0)]

In [ ]:
sisim_lifecyclem=sim_lifecycle[(sim_lifecycle.organization_id!=229) & (sim_lifecycle.organization_id!=406)]

In [ ]:
sim_lifecycle=sim_lifecycle[(sim_lifecycle.organization_id!=412) & (sim_lifecycle.organization_id!=1109)]

In [ ]:
sim_lifecycle=sim_lifecycle[(sim_lifecycle.organization_id!=229) & (sim_lifecycle.organization_id!=1066)]

In [ ]:
# join information at company level to information at sim level

to_activate=sim_lifecycle.merge(companies, on='organization_id')

In [ ]:
to_activate.assigned_month.isna().value_counts()

False    1014800
True      159931
Name: assigned_month, dtype: int64

In [ ]:

to_activate=to_activate[to_activate.activated_month.isna()]
to_activate.sim_id.count()

709723

In [ ]:
to_activate=to_activate[to_activate.organization_id.isna()==False]

In [ ]:
grouping=['assigned_month']
segment
grouping.extend(segment)
grouping

['assigned_month', 'vertical_group', 'country_group', 'initial_1k_sims']

In [ ]:
to_activate=to_activate.groupby(grouping).count()

In [ ]:
to_activate.reset_index(inplace=True)

In [ ]:
to_activate['segment']=to_activate[segment].apply(tuple, axis=1)

In [ ]:
# create a lint of all months we are interested in, to store all info about activated sims
month_list= pd.date_range('2019-10-01',end_date, freq='1M')-pd.offsets.MonthBegin(1)

### 1st activated SIMs from actual data

In [ ]:
#create dictionary where to store results of 1st ACTIVATED sims
activated={}
for i in to_activate.segment.unique():
    for month in month_list:
        activated[i,str(month.strftime( "%Y-%m-%d"))]=0

In [ ]:
activated

{(('Vertical Group 1', 'Country Group 1', False), '2019-10-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2019-11-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2019-12-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-01-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-02-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-03-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-04-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-05-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-06-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-07-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-08-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-09-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-10-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-11-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False)

In [ ]:
# create dictionary storing the sims assigned (but not activated) for each month in our month list

to_activate_dict={}
for s in to_activate.segment.unique():
    #print(segment)
    temp_segment=to_activate[to_activate.segment==s]
    for month in month_list:
        #print(month)
        i=temp_segment.index[temp_segment['assigned_month']==month].tolist()
        if len(i)>0:
            amount=to_activate.sim_id.iloc[i[0]]
            #print(amount)
            to_activate_dict[s,str(month.strftime( "%Y-%m-%d"))]=amount
        else:
            to_activate_dict[s,str(month.strftime( "%Y-%m-%d"))]=0

In [ ]:
to_activate_dict

{(('Vertical Group 1', 'Country Group 1', False), '2019-10-01'): 91262,
 (('Vertical Group 1', 'Country Group 1', False), '2019-11-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2019-12-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-01-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-02-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-03-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-04-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-05-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-06-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-07-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-08-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-09-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-10-01'): 0,
 (('Vertical Group 1', 'Country Group 1', False), '2020-11-01'): 27,
 (('Vertical Group 1', 'Country Group 1', F

In [ ]:
# getting activated sims

running_tot=0
#looping thorugh the potential sims to activate from the to_activate_dict
for k in to_activate_dict: 
    potential=to_activate_dict[k] #potential sims to activate

    #since for each month after assignement there is a differnt probability of activation we loop through months to get the right probability
    for month in range(len(probs_dict[k[0]])):
        #print('potential to activate',potential)

        try:
            p=probs_dict[k[0]][month] #retrieving the correct probability
            active=p['pct']*potential  #getting the number of activated sims for that month 
            print(active)
            #get extended date of sims activation
            date=k[1]
            print(date)
            new_date = datetime.strptime(date, '%Y-%m-%d').date() + relativedelta.relativedelta(months=month)
            print(new_date)
            activated[k[0], str(new_date)]+=active #adding the activated sims to the dictionary of sims activated in a specific month (extended date)
            running_tot+=active #adding the activated sims to the running total of activated sims

        except:
            active=0
        potential=potential-active #calculate the sims left for activation
        #print('activated',active)

0.0
2023-11-01
2024-04-01
0.0
2023-11-01
2024-05-01
0.0
2023-11-01
2024-06-01
0.0
2023-11-01
2024-07-01
0.0
2023-11-01
2024-08-01
0.0
2023-11-01
2024-09-01
0.0
2023-11-01
2024-10-01
0.0
2023-11-01
2024-11-01
0.0
2023-11-01
2024-12-01
0.0
2023-11-01
2025-01-01
0.0
2023-11-01
2025-02-01
0.0
2023-12-01
2023-12-01
0.0
2023-12-01
2024-01-01
0.0
2023-12-01
2024-02-01
0.0
2023-12-01
2024-03-01
0.0
2023-12-01
2024-04-01
0.0
2023-12-01
2024-05-01
0.0
2023-12-01
2024-06-01
0.0
2023-12-01
2024-07-01
0.0
2023-12-01
2024-08-01
0.0
2023-12-01
2024-09-01
0.0
2023-12-01
2024-10-01
0.0
2023-12-01
2024-11-01
0.0
2023-12-01
2024-12-01
0.0
2023-12-01
2025-01-01
0.0
2023-12-01
2025-02-01
0.0
2023-12-01
2025-03-01
0.0
2019-10-01
2019-11-01
0.0
2019-10-01
2019-12-01
0.0
2019-10-01
2020-01-01
0.0
2019-10-01
2020-02-01
0.0
2019-10-01
2020-03-01
0.0
2019-10-01
2020-04-01
0.0
2019-11-01
2019-12-01
0.0
2019-11-01
2020-01-01
0.0
2019-11-01
2020-02-01
0.0
2019-11-01
2020-03-01
0.0
2019-11-01
2020-04-01
0.0
2019-11-

In [ ]:
activated

{(('Vertical Group 1', 'Country Group 1', False),
  '2019-10-01'): 40.94555749802454,
 (('Vertical Group 1', 'Country Group 1', False),
  '2019-11-01'): 1920.2054550799064,
 (('Vertical Group 1', 'Country Group 1', False),
  '2019-12-01'): 3876.4147624425564,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-01-01'): 3243.170536998932,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-02-01'): 2564.0390488419916,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-03-01'): 1967.5046335690056,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-04-01'): 914.2154648266413,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-05-01'): 648.775298977359,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-06-01'): 468.05007967572607,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-07-01'): 1114.0015471015038,
 (('Vertical Group 1', 'Country Group 1', False),
  '2020-08-01'): 1356.1451026501784,
 (('Vertical Group 1', 'Country Group 1', False)

In [ ]:
#create dictionary of activated for the 1st time by month (totals, no segment splits)
first_act_tot={}
for m in month_list:
    m=datetime.strftime(m, '%Y-%m-%d')
    #print(type(m_str))
    first_act_tot[m]=0 
    for k in activated.keys():
        #print(type(k[1]))
        if k[1]==m:
            #print(activated.get(k))
            first_act_tot[m]+=activated.get(k) #update month total (no segment split)
first_act_tot

{'2019-10-01': 77.98723531194632,
 '2019-11-01': 1983.2494560949835,
 '2019-12-01': 3946.3511968352313,
 '2020-01-01': 3374.650074317927,
 '2020-02-01': 2653.1741747076267,
 '2020-03-01': 2105.0914383414074,
 '2020-04-01': 1091.2183020583605,
 '2020-05-01': 808.236300933428,
 '2020-06-01': 666.3945271155426,
 '2020-07-01': 1262.5585523432908,
 '2020-08-01': 1510.3673531024162,
 '2020-09-01': 1002.8306281554808,
 '2020-10-01': 773.6486699169964,
 '2020-11-01': 416.21449887193796,
 '2020-12-01': 650.362285636001,
 '2021-01-01': 249.30754983644727,
 '2021-02-01': 380.2125481268739,
 '2021-03-01': 373.9257987184425,
 '2021-04-01': 398.7535668565942,
 '2021-05-01': 753.9654936212625,
 '2021-06-01': 708.0435112347736,
 '2021-07-01': 694.6411251487987,
 '2021-08-01': 743.3672547600673,
 '2021-09-01': 765.9099289417048,
 '2021-10-01': 812.8617311686792,
 '2021-11-01': 877.8463445607152,
 '2021-12-01': 934.1637637206097,
 '2022-01-01': 1770.460872938296,
 '2022-02-01': 2313.2473125795386,
 '202

---

### 1st Activation SIMs for future months

In [ ]:
# future months on which to base the prediction

month_list_pred= pd.date_range(starting_date_for_pred,end_date, freq='1M')-pd.offsets.MonthBegin(1)

In [ ]:
#set how many sims are expected to be assigned per month

to_activate_pred={}
for i in to_activate.segment.unique():
    #add calculated proportion per each segment
    for month in month_list_pred:
        to_activate_pred[i,str(month.strftime( "%Y-%m-%d"))]=70000/24

In [ ]:
#use this to set allocation for specific segments

#i=('Vertical Group Other', 'Country Group Other', True)
#for month in month_list_pred:
#    to_activate_pred[i,str(month.strftime( "%Y-%m-%d"))]=(70000*0.8)

In [ ]:
to_activate_pred

{(('Vertical Group 1', 'Country Group 1', False),
  '2023-05-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', False),
  '2023-06-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', False),
  '2023-07-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', False),
  '2023-08-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', False),
  '2023-09-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', False),
  '2023-10-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', False),
  '2023-11-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', False),
  '2023-12-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', True),
  '2023-05-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', True),
  '2023-06-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', True),
  '2023-07-01'): 2916.6666666666665,
 (('Vertical Group 1', 'Country Group 1', True

In [ ]:
# emplty dictionary to store activated sims

activated_pred={}
for i in to_activate.segment.unique():
    for month in month_list_pred:
        activated_pred[i,str(month.strftime( "%Y-%m-%d"))]=0

In [ ]:
# getting activated sims for future months (by segment)

running_tot_pred=0
#looping thorugh the potential sims to activate from the to_activate_dict
for k in to_activate_pred: 
    potential=to_activate_pred[k] #potential sims to activate

    #since for each month after assignement there is a differnt probability of activation we loop through months to get the right probability
    for month in range(len(probs_dict[k[0]])):
        print('potential to activate',potential)

        try:
            p=probs_dict[k[0]][month] #retrieving the correct probability
            active=p['pct']*potential  #getting the number of activated sims for that month 
            #print(p,' ', active)
            #get extended date of sims activation
            date=k[1]
            new_date = datetime.strptime(date, '%Y-%m-%d').date() + relativedelta.relativedelta(months=month)

            print(new_date)
            activated_pred[k[0], str(new_date)]+=active #adding the activated sims to the dictionary of sims activated in a specific month (extended date)
            running_tot+=active #adding the activated sims to the running total of activated sims

        except:
            active=0
        potential=potential-active #calculate the sims left for activation
        #print('activated',active)

2024-04-01
potential to activate 2723.34699124287
2024-05-01
potential to activate 2723.34699124287
2024-06-01
potential to activate 2723.34699124287
2024-07-01
potential to activate 2723.34699124287
potential to activate 2723.34699124287
2024-09-01
potential to activate 2723.34699124287
2024-10-01
potential to activate 2723.34699124287
potential to activate 2723.34699124287
2024-12-01
potential to activate 2723.34699124287
2025-01-01
potential to activate 2723.34699124287
potential to activate 2916.6666666666665
2023-12-01
potential to activate 2874.4878283923836
2024-01-01
potential to activate 2874.4878283923836
2024-02-01
potential to activate 2874.4878283923836
2024-03-01
potential to activate 2874.4878283923836
2024-04-01
potential to activate 2874.4878283923836
2024-05-01
potential to activate 2874.4878283923836
2024-06-01
potential to activate 2874.4878283923836
2024-07-01
potential to activate 2874.4878283923836
2024-08-01
potential to activate 2874.4878283923836
potential to 

In [ ]:
# dictionary of activated sims for future months allocation
pred_first_act_tot={}
pred_first_act_segm={}
for m in month_list_pred:
    m=datetime.strftime(m, '%Y-%m-%d')
    pred_first_act_tot[m]=0 
    for s in to_activate.segment.unique():
        #print(s)
        pred_first_act_segm[s,m]=0
    for k in activated_pred.keys():
        if k[1] ==m:
            pred_first_act_tot[m]+=activated_pred.get(k) #update month total (no segment split)
            pred_first_act_segm[s,m]+=activated_pred.get(k) 
pred_first_act_tot

{'2023-05-01': 1725.4770967278105,
 '2023-06-01': 5422.5124002221955,
 '2023-07-01': 9811.927120377712,
 '2023-08-01': 14585.390288231907,
 '2023-09-01': 17905.052010929936,
 '2023-10-01': 20080.848747251624,
 '2023-11-01': 22196.983045592715,
 '2023-12-01': 23645.556755749916}

In [ ]:
# summing together 1st activation from current data + 1st activation from assumed future activations

for k in activated_pred.keys():
    try:
        #print(activated.get(k))
        #print(activated_pred.get(k))
        activated[k]=activated.get(k)+activated_pred.get(k)
    except:
        print('something went wrong') 

In [ ]:
# generate dataframe for 1st activation on actual data

new_month1 = (pd.DataFrame.from_dict(activated, orient='index')
                  .rename(columns=lambda x: x)
                  .add_prefix('sim_id')
                  .rename_axis('date_segment')
                  .reset_index())

In [ ]:
# generate dataframe for 1st activation on assumed future activations

new_month2 = (pd.DataFrame.from_dict(activated_pred, orient='index')
                  .rename(columns=lambda x: x)
                  .add_prefix('sim_id')
                  .rename_axis('date_segment')
                  .reset_index())

In [ ]:
# merge the 2 dataframes
new=new_month1.merge(new_month2, on='date_segment', how='left')

In [ ]:
new=new.fillna(0)

In [ ]:
# get total number of 1st activated sims by summing the 2 dataframes (1st activation from current data + 1st activation from assumed future activations)
new['sim_id']= new.sim_id0_x+new.sim_id0_y

In [ ]:
activation_df=new[['date_segment','sim_id']]

In [ ]:
activation_df[['segment','month']] = pd.DataFrame(activation_df.date_segment.tolist(), index= activation_df.index)


/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
activation_df[activation_df['month']=='2023-05-01']

,date_segment,sim_id,segment,month
43,"((Vertical Group 1, Country Group 1, False), 2...",293.779920,"(Vertical Group 1, Country Group 1, False)",2023-05-01
94,"((Vertical Group 1, Country Group 1, True), 20...",112.834804,"(Vertical Group 1, Country Group 1, True)",2023-05-01
145,"((Vertical Group 2, Country Group 1, False), 2...",5358.271990,"(Vertical Group 2, Country Group 1, False)",2023-05-01
196,"((Vertical Group 2, Country Group 1, True), 20...",580.917798,"(Vertical Group 2, Country Group 1, True)",2023-05-01
247,"((Vertical Group 3, Country Group 1, True), 20...",211.590810,"(Vertical Group 3, Country Group 1, True)",2023-05-01
298,"((Vertical Group 3, Country Group Other, True)...",187.367720,"(Vertical Group 3, Country Group Other, True)",2023-05-01
349,"((Vertical Group Other, Country Group 1, False...",484.256370,"(Vertical Group Other, Country Group 1, False)",2023-05-01
400,"((Vertical Group Other, Country Group 1, True)...",250.204505,"(Vertical Group Other, Country Group 1, True)",2023-05-01
451,"((Vertical Group 2, Country Group Other, True)...",390.234017,"(Vertical Group 2, Country Group Other, True)",2023-05-01
502,"((Vertical Group Other, Country Group 2, True)...",255.641577,"(Vertical Group Other, Country Group 2, True)",2023-05-01


In [ ]:
# keep only month for which prediction is to be made

activation_df=activation_df[activation_df.month>='2023-05-01']

---

### Create data frame combining 1st activations and returning sims

In [ ]:
activation_df.month=activation_df.month.apply(pd.to_datetime)

In [ ]:
activation_df.month = pd.to_datetime(activation_df.month).dt.date


In [ ]:
final_df=active_df.merge(activation_df, on=['month','segment'],how='outer')

In [ ]:
# total number of active sims as sum of returning sims and 1st activation sims
final_df.fillna(0, inplace=True)
final_df['sim_id']=final_df.sim_id_x+final_df.sim_id_y

In [ ]:
# rename columns
final_df.rename(columns={'sim_id_x':'returning', 'sim_id_y':'fst_activation'}, inplace=True)

In [ ]:
final_df

,month,vertical_group,country_group,initial_1k_sims,organization_id,returning,assigned_month,activated_month,deployed_month,previous_month_online,...,active_SIMs_12,mrr_3,mrr_6,mrr_9,mrr_12,segment,month_diff,date_segment,fst_activation,sim_id
0,2019-02-01,Vertical Group 1,Country Group 1,True,3.0,3.0,3.0,3.0,0.0,0.0,...,3.0,3.0,3.0,3.0,3.0,"(Vertical Group 1, Country Group 1, True)",51.0,0,0.000000,3.000000
1,2019-02-01,Vertical Group 3,Country Group Other,True,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,"(Vertical Group 3, Country Group Other, True)",51.0,0,0.000000,1.000000
2,2019-02-01,Vertical Group Other,Country Group 1,True,3.0,3.0,3.0,3.0,3.0,0.0,...,3.0,1.0,3.0,3.0,3.0,"(Vertical Group Other, Country Group 1, True)",51.0,0,0.000000,3.000000
3,2019-03-01,Vertical Group 1,Country Group 1,False,18.0,18.0,18.0,18.0,6.0,0.0,...,18.0,18.0,18.0,18.0,18.0,"(Vertical Group 1, Country Group 1, False)",50.0,0,0.000000,18.000000
4,2019-03-01,Vertical Group 1,Country Group 1,True,7.0,7.0,7.0,7.0,1.0,3.0,...,7.0,7.0,7.0,7.0,7.0,"(Vertical Group 1, Country Group 1, True)",50.0,0,0.000000,7.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,2023-08-01,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,"(Vertical Group 1, Country Group 2, False)",0.0,"((Vertical Group 1, Country Group 2, False), 2...",1303.921569,1303.921569
989,2023-09-01,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,"(Vertical Group 1, Country Group 2, False)",0.0,"((Vertical Group 1, Country Group 2, False), 2...",1944.444444,1944.444444
990,2023-10-01,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,"(Vertical Group 1, Country Group 2, False)",0.0,"((Vertical Group 1, Country Group 2, False), 2...",2939.542484,2939.542484
991,2023-11-01,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,"(Vertical Group 1, Country Group 2, False)",0.0,"((Vertical Group 1, Country Group 2, False), 2...",3311.274510,3311.274510


In [ ]:
#keeping only relevant columns 

final_df=final_df[['month','segment','month_diff','sim_id','returning','fst_activation','months_since_last_online']]

In [ ]:
final_df.head()

,month,segment,month_diff,sim_id,returning,fst_activation,months_since_last_online
0,2019-02-01,"(Vertical Group 1, Country Group 1, True)",51.0,3.0,3.0,0.0,0.0
1,2019-02-01,"(Vertical Group 3, Country Group Other, True)",51.0,1.0,1.0,0.0,0.0
2,2019-02-01,"(Vertical Group Other, Country Group 1, True)",51.0,3.0,3.0,0.0,0.0
3,2019-03-01,"(Vertical Group 1, Country Group 1, False)",50.0,18.0,18.0,0.0,0.0
4,2019-03-01,"(Vertical Group 1, Country Group 1, True)",50.0,7.0,7.0,0.0,3.0


In [ ]:
final_df.month = pd.to_datetime(final_df.month).dt.date


/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
final_df

,month,segment,month_diff,sim_id,returning,fst_activation,months_since_last_online
0,2019-02-01,"(Vertical Group 1, Country Group 1, True)",51.0,3.000000,3.0,0.000000,0.0
1,2019-02-01,"(Vertical Group 3, Country Group Other, True)",51.0,1.000000,1.0,0.000000,0.0
2,2019-02-01,"(Vertical Group Other, Country Group 1, True)",51.0,3.000000,3.0,0.000000,0.0
3,2019-03-01,"(Vertical Group 1, Country Group 1, False)",50.0,18.000000,18.0,0.000000,0.0
4,2019-03-01,"(Vertical Group 1, Country Group 1, True)",50.0,7.000000,7.0,0.000000,3.0
...,...,...,...,...,...,...,...
988,2023-08-01,"(Vertical Group 1, Country Group 2, False)",0.0,1303.921569,0.0,1303.921569,0.0
989,2023-09-01,"(Vertical Group 1, Country Group 2, False)",0.0,1944.444444,0.0,1944.444444,0.0
990,2023-10-01,"(Vertical Group 1, Country Group 2, False)",0.0,2939.542484,0.0,2939.542484,0.0
991,2023-11-01,"(Vertical Group 1, Country Group 2, False)",0.0,3311.274510,0.0,3311.274510,0.0


In [ ]:
# total returning for the last month of actual data
final_df[final_df.month==starting_date_for_pred].sim_id.sum()

224924.1501471321

---

## Returning SIMs for future months

Here some of the steps above are wrapped into function (to improve code quality it could make sense to define these functions at the beginning and then call them)

In [ ]:
# define range of future months for which to calculate returning sims
month_list_pred= pd.date_range(date_for_pred,end_date, freq='1M')-pd.offsets.MonthBegin(1)


In [ ]:
# function creating dictionary where to store returnig sims

def create_returning(date_for_pred):
        returning={}
        for i in final.segment.unique():
                returning[str(date_for_pred),i]=0.0
        return returning

In [ ]:
# define function to calculate returning sims

def get_returning_sims(returning,return_pct,date_for_pred):
    total=0
    for month in range(1,12):
        temp=final_df[final_df.month_diff==month]
        for segment in final_df.segment.unique(): #doing the calculation for each segment
            temp_segment=temp[temp.segment==segment]
            #print(temp_segment)
            try:
                sims= float(temp_segment.sim_id)
            except:
                sims=0
        #print(sims)
            pct=return_pct[month,segment]
        #print(pct)
            if np.isnan(pct):
                pct=0
            ret=sims*pct
            #print(ret)
        #if month==1:
            returning[str(date_for_pred),segment]+=ret

            total+=ret
    return returning, total

In [ ]:
# function to calculate n. of months between date and prediction date

def count_months(date_for_pred,date):
    r = relativedelta.relativedelta(date_for_pred,date)    
    months = (r.years * 12) + r.months
    return months

In [ ]:
# function used to transform the dictionaries with retuning sims data into a dataframe

def create_new_month_df(returning):
    new_added_month=pd.DataFrame.from_dict(returning, orient='index')
    new_added_month = (pd.DataFrame.from_dict(returning, orient='index')
                  .rename(columns=lambda x: x)
                  .add_prefix('sim_id')
                  .rename_axis('date_segment')
                  .reset_index())
    new_added_month[['month','segment']] = pd.DataFrame(new_added_month.date_segment.tolist(), index= new_added_month.index)
    new_added_month.rename(columns={'sim_id0':'sim_id'}, inplace=True)
    new_added_month.drop(['date_segment'], axis=1, inplace=True)

    new_added_month.month=new_added_month.month.apply(pd.to_datetime)
    new_added_month.month = pd.to_datetime(new_added_month.month).dt.date

    new_added_month.rename(columns={'sim_id':'new_returning'}, inplace=True)
    return new_added_month

In [ ]:
# function used to add the newly calculated data on returning sims to the existing dataframe

def update_final(final_df, new_added_month):
    final_df=final_df.merge(new_added_month, on=['month','segment'],how='left')
    final_df=final_df.fillna(0)
    final_df['sim_id']=final_df.sim_id+final_df.new_returning
    final_df['returning']=final_df.returning+final_df.new_returning
    final_df.drop(['new_returning'],axis=1, inplace=True)
    return final_df

In [ ]:
# function to print summary of each future month
def get_report(mo,final_df):
    returning_sims=final_df[final_df.month==mo].returning.sum()
    fst=final_df[final_df.month==mo].fst_activation.sum()
    print('Summary of the month', mo)
    print('Number of returning sims', returning_sims)
    print('Number of fst activations', fst)
    print('Total active sims in the month', returning_sims+fst)


In [ ]:
# perform the process for each month in the month_list_pred list

for mo in month_list_pred:
    print((mo.strftime( "%Y-%m-%d")))
    #final_df.month=final_df.month.apply(pd.to_datetime)
    final_df['month_diff'] = final_df.month.apply(lambda x:count_months(mo,x))
    #print(final_df.head())
    returning=create_returning(mo)
    returning, total= get_returning_sims(returning,return_pct,mo)
    #print(returning)

    new_added_month=create_new_month_df(returning)
    #print(new_added_month.head())
    final_df=update_final(final_df,new_added_month)
    #print(final_df.shape)
    get_report(mo,final_df)



/tmp/ipykernel_1557/1665133270.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['month_diff'] = final_df.month.apply(lambda x:count_months(mo,x))
2023-06-01
Summary of the month 2023-06-01 00:00:00
Number of returning sims 204777.68656411607
Number of fst activations 21201.807223800904
Total active sims in the month 225979.49378791696
2023-07-01
Summary of the month 2023-07-01 00:00:00
Number of returning sims 205934.5268577861
Number of fst activations 31319.063680410927
Total active sims in the month 237253.59053819705
2023-08-01
Summary of the month 2023-08-01 00:00:00
Number of returning sims 215081.8413559154
Number of fst activations 42670.45442262026
Total active sims in the month 257752.29577853568
2023-09-01
Summary of the mo

In [ ]:
final_df.head()

,month,segment,month_diff,sim_id,returning,fst_activation,months_since_last_online
0,2019-02-01,"(Vertical Group 1, Country Group 1, True)",58,3.0,3.0,0.0,0.0
1,2019-02-01,"(Vertical Group 3, Country Group Other, True)",58,1.0,1.0,0.0,0.0
2,2019-02-01,"(Vertical Group Other, Country Group 1, True)",58,3.0,3.0,0.0,0.0
3,2019-03-01,"(Vertical Group 1, Country Group 1, False)",57,18.0,18.0,0.0,0.0
4,2019-03-01,"(Vertical Group 1, Country Group 1, True)",57,7.0,7.0,0.0,3.0


use these blocks in case you want to get info about active sims for specific months

In [ ]:
final_df[final_df.month==date_for_pred].sim_id.sum()

225979.49378791696

In [ ]:
final_df[final_df.month==date_for_pred].returning.sum()

204777.68656411607

In [ ]:
final_df[final_df.month==date_for_pred].fst_activation.sum()

21201.807223800904

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d15c6eac-0104-459f-ae67-fb12fbc22d47' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>